In [1]:
# New notebook for heatmaps, based on daily data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import seaborn as sns

import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


In [2]:
dashDir = 'ssi_dashboard/'

rootdir = os.getcwd() +"\\" + dashDir


# for subdir, dirs, files in os.walk(rootdir):
#     print(dirs)
#     if not len(dirs) == 0:
#         latestdir = subdir
#         # latestDate = pd.to_datetime(subdir[-10:])
#     # print(files)
#     # print(dirs[-1])

for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])

latestDate

Timestamp('2021-11-01 00:00:00')

In [3]:
curFileName = '\\Regionalt_DB\\05_bekraeftede_tilfaelde_doede_pr_region_pr_alders_grp.csv'
alldfs = []

# 04_indlagte_pr_alders_grp_pr_region


allDates = []


for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    try:
        curDate = pd.to_datetime(curPath[-10:])

        
        curcsvpath = curPath + curFileName
        curdf = pd.read_csv(curcsvpath,delimiter = ';',dtype=str,encoding='latin-1')

        allDates.append(curDate)
        alldfs.append(curdf)
    except:
        print(curPath)
    
print(f'Number of dataframes in alldfs: {len(alldfs)}')


c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_dashboard/SSI_dashboard_2021-05-25
Number of dataframes in alldfs: 161


In [4]:
numDataframe = len(alldfs)

# Whole country, age-split
allAge = curdf['Alders gruppe'].unique()

dfCase = pd.DataFrame()
dfDeath = pd.DataFrame()


dfCase['Dato'] = allDates
dfDeath['Dato'] = allDates
# fullDates = np.arange(allDates[0],allDates[-1],np.timedelta64(1,'D'))
# dfCase['Dato'] = fullDates
# dfDeath['Dato'] = fullDates


for ag in range(len(allAge)):
    curAge = allAge[ag]

    allCase = []
    allDeaths = []

    prevDate = allDates[0]

    for da in range(0,numDataframe):
        thisdf = alldfs[da]
        thisdf = thisdf[thisdf['Alders gruppe'] == curAge]

        thisCase = thisdf['Bekræftede tilfælde'].astype(int).sum()
        thisDeaths = thisdf['Døde'].astype(int).sum()


        allCase.append(thisCase)
        allDeaths.append(thisDeaths) 

        # curDate = allDates[da]

        # dateDiff = curDate - prevDate
        # dateDiff = int((dateDiff/np.timedelta64(1,'D')))

        # if  (dateDiff == 1):
        #     allCase.append(thisCase)
        #     allDeaths.append(thisDeaths) 
        # elif (dateDiff == 2):
        #     allCase.append(thisCase/2)
        #     allCase.append(thisCase/2)
        #     allDeaths.append(thisDeaths/2)
        #     allDeaths.append(thisDeaths/2)
        # elif (dateDiff == 3):
        #     allCase.append(thisCase/3)
        #     allCase.append(thisCase/3)
        #     allCase.append(thisCase/3)
        #     allDeaths.append(thisDeaths/3)
        #     allDeaths.append(thisDeaths/3)
        #     allDeaths.append(thisDeaths/3)
        # elif (dateDiff == 4):
        #     allCase.append(thisCase/4)
        #     allCase.append(thisCase/4)
        #     allCase.append(thisCase/4)
        #     allCase.append(thisCase/4)
        #     allDeaths.append(thisDeaths/4)
        #     allDeaths.append(thisDeaths/4)
        #     allDeaths.append(thisDeaths/4)
        #     allDeaths.append(thisDeaths/4)
        # else:
        #     # print(dateDiff)


        # prevDate = curDate 

    
    dfCase[curAge] = allCase
    dfDeath[curAge] = allDeaths

# dfCase
# thisdf

In [5]:
# dfDeath

In [6]:
# allCase

In [7]:
# Calculate difference, and average out weekends
# dfDeath.diff().iloc[1:]

dfCaseDiff = dfCase.diff().iloc[1:]
dfDeathDiff = dfDeath.diff().iloc[1:]
# dfCaseDiff

actualDiffDates = dfCaseDiff.Dato
actualDiffDates = (actualDiffDates/np.timedelta64(1,'D')).astype(int)

# actualDiffDates

fullDates = np.arange(allDates[0],allDates[-1],np.timedelta64(1,'D'))

dfCaseDiffCorr = dfCaseDiff.copy()
dfDeathDiffCorr = dfDeathDiff.copy()

for r in range(0,len(dfCaseDiff)):
    if ((actualDiffDates.values[r]) != 1):
        # print(dfCaseDiff.iloc[r])
        curRow = dfCaseDiff.iloc[r]
        numDays = int(curRow.Dato/np.timedelta64(1,'D'))

        rowToAdd = curRow/numDays

        for k in range(1,numDays+1):
            dfCaseDiffCorr.loc[r+(0.1*k)] = rowToAdd
        dfCaseDiffCorr = dfCaseDiffCorr.drop(r+1)
        
        # And deaths
        curRow = dfDeathDiff.iloc[r]
        numDays = int(curRow.Dato/np.timedelta64(1,'D'))

        rowToAdd = curRow/numDays

        for k in range(1,numDays+1):
            dfDeathDiffCorr.loc[r+(0.1*k)] = rowToAdd
        dfDeathDiffCorr = dfDeathDiffCorr.drop(r+1)

        # newdf = pd.concat([dfCaseDiff.iloc[:2], curRow, dfCaseDiff.iloc[2:]]).reset_index(drop=True)
        


dfCaseDiffCorr = dfCaseDiffCorr.sort_index().reset_index(drop=True)
dfDeathDiffCorr = dfDeathDiffCorr.sort_index().reset_index(drop=True)
# dfCaseDiffCorr
dfCaseDiffCorr = dfCaseDiffCorr.iloc[:,1:]
dfDeathDiffCorr = dfDeathDiffCorr.iloc[:,1:]

In [8]:
dfCaseDiffCorrToSave = dfCaseDiffCorr
dfCaseDiffCorrToSave['Dato'] = fullDates
dfCaseDiffCorrToSave.to_csv('DKDailyAgeCase.csv',index=False)
dfDeathDiffCorrToSave = dfDeathDiffCorr
dfDeathDiffCorrToSave['Dato'] = fullDates
dfDeathDiffCorrToSave.to_csv('DKDailyAgeDeath.csv',index=False)

In [9]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

cmap = cm.get_cmap('twilight',len(allAge)+1)

for k in range(0,len(allAge)):
    ag = allAge[k]
    curVals = dfCaseDiffCorr[ag].values
    ax1.plot(fullDates,curVals,'.:',color=cmap(k),linewidth=0.5,markersize=2)
    ax1.plot(rnTime(fullDates,7),rnMean(curVals,7),color=cmap(k),label=ag)
    
    curVals = dfDeathDiffCorr[ag].values
    ax2.plot(fullDates,curVals,'.:',color=cmap(k),linewidth=0.5,markersize=2)
    ax2.plot(rnTime(fullDates,7),rnMean(curVals,7),color=cmap(k),label=ag)

    

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey')

ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey',label='Weekend') 
ax1.set_xlim(left=fullDates[3])
ax1.set_xlim(right=fullDates[-1])

ax1.legend(loc='upper left',fontsize=8)

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')

cmap = cm.get_cmap('Spectral',len(allAge)+1)

newVals = dfCaseDiffCorr['0-9'].values
curVals = np.zeros(newVals.shape)
prevVals = np.zeros(newVals.shape)
curValsDe = np.zeros(newVals.shape)
prevValsDe = np.zeros(newVals.shape)

for k in range(0,len(allAge)):
    ag = allAge[k]
    newVals = dfCaseDiffCorr[ag].values
    newValsDe = dfDeathDiffCorr[ag].values

    curVals = curVals + newVals
    curValsDe = curValsDe + newValsDe

    # ax1.fill_between(fullDates,curVals,prevVals,color=cmap(k))
    # ax1.fill_between(rnTime(fullDates,7),rnMean(newVals,7),rnMean(prevVals,7),color=cmap(k),label=ag)
    ax1.fill_between(rnTime(fullDates,7),rnMean(curVals,7),rnMean(prevVals,7),color=cmap(k),label=ag)
    # ax1.plot(fullDates,curVals,'.:',color=cmap(k),linewidth=0.5,markersize=2)
    # ax1.plot(rnTime(fullDates,7),rnMean(curVals,7),color=cmap(k),label=ag)
    
    # curVals = dfDeathDiffCorr[ag].values
    # ax2.plot(fullDates,curVals,'.:',color=cmap(k),linewidth=0.5,markersize=2)
    # ax2.plot(rnTime(fullDates,7),rnMean(curVals,7),color=cmap(k),label=ag)
    ax2.fill_between(rnTime(fullDates,7),rnMean(curValsDe,7),rnMean(prevValsDe,7),color=cmap(k),label=ag)


    prevVals += newVals
    prevValsDe += newValsDe
    

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')

ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey',label='Weekend') 
ax1.set_xlim(left=fullDates[3])
ax1.set_xlim(right=fullDates[-1])

ax1.legend(loc='upper left',fontsize=8)

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# dfCaseDiffCorr.iloc[:,1:]

In [12]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

bgColor = 'lightgrey'
fig.patch.set_facecolor(bgColor)
ax1.set_facecolor(bgColor)
ax2.set_facecolor(bgColor)

newVals = dfCaseDiffCorr['0-9'].values
curVals = np.zeros(newVals.shape)
prevVals = np.zeros(newVals.shape)

# totCase = dfCaseDiffCorr.iloc[:,1:].sum(axis=1)
totCase = dfCaseDiffCorr.iloc[:,:-1].sum(axis=1)
totCaseRM = rnMean(totCase,7)

ax1.plot(rnTime(fullDates,7),totCaseRM,'k',linewidth=1,label='Total')

firstDateToShow = np.datetime64('2021-04-01')

cmap = cm.get_cmap('Spectral',len(allAge)+1)
for k in range(0,len(allAge)):
    ag = allAge[k]
    newVals = dfCaseDiffCorr[ag].values

    curVals = curVals + newVals

    ax1.fill_between(rnTime(fullDates,7),rnMean(curVals,7),rnMean(prevVals,7),color=cmap(k),label=ag)
    
    ax2.fill_between(rnTime(fullDates,7),100*np.divide(rnMean(curVals,7),totCaseRM),100*np.divide(rnMean(prevVals,7),totCaseRM),color=cmap(k),label=ag)

    # ax2.text(firstDateToShow,(100/totCase[0])*(prevVals[0]),ag)
    # ax1.text(firstDateToShow,prevVals[0]+curVals[0]/4,ag)

    prevVals += newVals
    

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')

ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey',label='Weekend') 
# ax1.set_xlim(left=fullDates[3])
ax1.set_xlim(left=firstDateToShow)
ax1.set_xlim(right=fullDates[-1])

ax1.set_ylabel('Tilfælde')
ax2.set_ylabel('Andel ud af alle tilfælde [%]')

ax2.grid(axis='y')

ax1.legend(loc='upper left',fontsize=8)

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesAgeSplit')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# pd.to_datetime(daysToUse).strftime('%d.%B')
# # daysToUse

In [14]:
fig,ax1 = plt.subplots()

firstDataPoint = 28
dfToPlot = dfCaseDiffCorr.iloc[firstDataPoint:,:-1]
daysToUse = fullDates[firstDataPoint:]

# plt.pcolor(dfCaseDiffCorr.iloc[:,1:])
# dfCaseDiffCorr 
# dfCaseDiffCorr.heatmap()
g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %b'))
# g = sns.heatmap(dfToPlot.transpose())
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %B'), fontsize =10)
# g = sns.heatmap(dfCaseDiffCorr,annot=True,fmt="1",cmap='turbo',xticklabels=pd.to_datetime(fullDates).strftime('%d-%b')[1:],yticklabels=dfCaseDiffCorr.columns[:-1])
# g = sns.heatmap(dfCaseDiffCorr,annot=True,fmt="1",cmap='turbo',xticklabels=pd.to_datetime(fullDates).strftime('%d-%b')[1:],yticklabels=dfCaseDiffCorr.columns[:-1])
g.invert_yaxis()

plt.title('Daglige tilfælde')

if saveFigures:
    plt.savefig('figs/HeatmapCasesNew')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
dfAges = pd.read_excel('DKfolketal2021clean.xlsx')
DK_age = []
DK_age.append(dfAges.iloc[0:10].Antal.sum())
DK_age.append(dfAges.iloc[10:20].Antal.sum())
DK_age.append(dfAges.iloc[20:30].Antal.sum())
DK_age.append(dfAges.iloc[30:40].Antal.sum())
DK_age.append(dfAges.iloc[40:50].Antal.sum())
DK_age.append(dfAges.iloc[50:60].Antal.sum())
DK_age.append(dfAges.iloc[60:70].Antal.sum())
DK_age.append(dfAges.iloc[70:80].Antal.sum())
DK_age.append(dfAges.iloc[80:90].Antal.sum())
DK_age.append(dfAges.iloc[90:].Antal.sum())
DK_age = np.array(DK_age)

# caseValsNorm = dfCase.iloc[:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
# caseValsNorm = 1000* np.transpose(caseValsNorm)

In [16]:
normDF = 10000 * dfCaseDiffCorr.iloc[:,:-1].div(DK_age)

fig,ax1 = plt.subplots()

firstDataPoint = 28
dfToPlot = normDF.iloc[firstDataPoint:]
daysToUse = fullDates[firstDataPoint:]

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'))
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %B'), fontsize =10)
g.invert_yaxis()


plt.title('Daglige tilfælde, per 10.000')

if saveFigures:
    plt.savefig('figs/HeatmapCasesNormNew')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:

firstDataPoint = 25
dfToPlot = dfCaseDiffCorr.iloc[firstDataPoint:,:-1]
daysToUse = fullDates[firstDataPoint:]


dfCaseDiffCorrRM = pd.DataFrame()

meanWidth = 7
for curCol in dfCaseDiffCorr.columns[:-1]:
    dfCaseDiffCorrRM[curCol] = rnMean(dfCaseDiffCorr[curCol],meanWidth)
# dfCaseDiffCorrRM[curCol]
dfToPlot = dfCaseDiffCorrRM.iloc[firstDataPoint:]

fig,ax1 = plt.subplots()

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %b'), fontsize =10)
g.invert_yaxis()

plt.title(f'Daglige tilfælde, {meanWidth} dages gennemsnit')

if saveFigures:
    plt.savefig(f'figs/HeatmapCasesNew_{meanWidth}rm')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:

firstDataPoint = 25
dfToPlot = normDF.iloc[firstDataPoint:,:-1]
daysToUse = fullDates[firstDataPoint:]

normDFRM = pd.DataFrame()

meanWidth = 7
for curCol in normDF.columns[:-1]:
    normDFRM[curCol] = rnMean(normDF[curCol],meanWidth)
# normDFRM[curCol]
dfToPlot = normDFRM.iloc[firstDataPoint:]

fig,ax1 = plt.subplots()

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %b'), fontsize =10)
g.invert_yaxis()

plt.title(f'Daglige tilfælde, per 10.000, {meanWidth} dages gennemsnit')

if saveFigures:
    plt.savefig(f'figs/HeatmapCasesNormNew_{meanWidth}rm')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:


# fig,ax1 = plt.subplots()

# cmap = cm.get_cmap('twilight',len(allAge)+1)

# # for ag in allAge:
# for k in range(0,len(allAge)):
#     ag = allAge[k]
#     curVals = dfCaseDiff[ag].values
#     ax1.plot(diffDates,curVals,'.:',color=cmap(k),linewidth=0.5,markersize=2)
#     ax1.plot(rnTime(diffDates,7),rnMean(curVals,7),color=cmap(k),label=ag)

#     # prevVals = curVals

# # ax1.plot(diffDates,dfCaseDiff,'-*')
# # ax1.plot(rnTime(diffDates,7),rnMean(dfCaseDiff,7),'-*')

# ax1.legend()
# plt.tight_layout()

# Hospital admissions

In [20]:
curFileName = '\\Regionalt_DB\\04_indlagte_pr_alders_grp_pr_region.csv'
alldfs_adm = []



allDates_adm = []


for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    try:
        curDate = pd.to_datetime(curPath[-10:])

        
        curcsvpath = curPath + curFileName
        curdf = pd.read_csv(curcsvpath,delimiter = ';',dtype=str,encoding='latin-1')

        allDates_adm.append(curDate)
        alldfs_adm.append(curdf)
    except:
        print(curPath)
    
print(f'Number of dataframes in alldfs_adm: {len(alldfs_adm)}')


c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData\ssi_dashboard/SSI_dashboard_2021-05-25
Number of dataframes in alldfs_adm: 161


In [21]:
numDataframe = len(alldfs_adm)

# Whole country, age-split

allAge = alldfs_adm[0]['Alders gruppe'].unique()

dfAdm = pd.DataFrame()

dfAdm['Dato'] = allDates_adm


for ag in range(len(allAge)):
    curAge = allAge[ag]

    allAdm = []

    prevDate = allDates[0]

    for da in range(0,numDataframe):
        thisdf = alldfs_adm[da]
        thisdf = thisdf[thisdf['Alders gruppe'] == curAge]

        thisAdm = thisdf['Indlæggelser'].astype(int).sum()


        allAdm.append(thisAdm)


    
    dfAdm[curAge] = allAdm

In [22]:
# Calculate difference, and average out weekends

dfAdmDiff = dfAdm.diff().iloc[1:]

actualDiffDates = dfAdmDiff.Dato
actualDiffDates = (actualDiffDates/np.timedelta64(1,'D')).astype(int)


fullDatesAdm = np.arange(allDates_adm[0],allDates_adm[-1],np.timedelta64(1,'D'))

dfAdmDiffCorr = dfAdmDiff.copy()

for r in range(0,len(dfAdmDiff)):
    if ((actualDiffDates.values[r]) != 1):
        # print(dfAdmDiff.iloc[r])
        curRow = dfAdmDiff.iloc[r]
        numDays = int(curRow.Dato/np.timedelta64(1,'D'))

        rowToAdd = curRow/numDays

        for k in range(1,numDays+1):
            dfAdmDiffCorr.loc[r+(0.01*k)] = rowToAdd
        dfAdmDiffCorr = dfAdmDiffCorr.drop(r+1)
        


dfAdmDiffCorr = dfAdmDiffCorr.sort_index().reset_index(drop=True)
dfAdmDiffCorr = dfAdmDiffCorr.iloc[:,1:]

In [23]:

sumAdm = dfAdmDiffCorr.sum(axis=1)
sumAdm

fig,ax1 = plt.subplots()


cmap = cm.get_cmap('tab20c',len(allAge))
for k in range(len(allAge)):
    curAge = allAge[k]

    ax1.plot(fullDatesAdm,dfAdmDiffCorr[curAge],'.:',markersize=2,linewidth=0.5,color=cmap(k))
    ax1.plot(rnTime(fullDatesAdm,7),rnMean(dfAdmDiffCorr[curAge],7),color=cmap(k),label=curAge)


ax1.plot(fullDatesAdm,sumAdm,'.:',markersize=2,linewidth=0.5,color='k')
ax1.plot(rnTime(fullDatesAdm,7),rnMean(sumAdm,7),color='k',label='Total')

ax1.legend()
ax1.set_ylim(bottom=0)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# dfAdmDiffCorr

In [51]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

bgColor = 'lightgrey'
fig.patch.set_facecolor(bgColor)
ax1.set_facecolor(bgColor)
ax2.set_facecolor(bgColor)

newVals = dfAdmDiffCorr['0-9'].values
curVals = np.zeros(newVals.shape)
prevVals = np.zeros(newVals.shape)

# totCase = dfAdmDiffCorr.iloc[:,1:].sum(axis=1)
totCase = dfAdmDiffCorr.sum(axis=1)
totCaseRM = rnMean(totCase,7)

ax1.plot(rnTime(fullDates,7),totCaseRM,'k',linewidth=1,label='Total')

firstDateToShow = np.datetime64('2021-05-01')

cmap = cm.get_cmap('Spectral',len(allAge)+1)
for k in range(0,len(allAge)):
    ag = allAge[k]
    newVals = dfAdmDiffCorr[ag].values

    curVals = curVals + newVals

    ax1.fill_between(rnTime(fullDates,7),rnMean(curVals,7),rnMean(prevVals,7),color=cmap(k),label=ag)
    
    ax2.fill_between(rnTime(fullDates,7),100*np.divide(rnMean(curVals,7),totCaseRM),100*np.divide(rnMean(prevVals,7),totCaseRM),color=cmap(k),label=ag)

    # ax2.text(firstDateToShow,(100/totCase[0])*(prevVals[0]),ag)
    # ax1.text(firstDateToShow,prevVals[0]+curVals[0]/4,ag)

    prevVals += newVals
    

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')

ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey',label='Weekend') 
# ax1.set_xlim(left=fullDates[3])
ax1.set_xlim(left=firstDateToShow)
ax1.set_xlim(right=fullDates[-1])

ax1.set_ylabel('Nyindlæggelser')
ax2.set_ylabel('Andel ud af alle nyindlæggelser [%]')

ax1.legend(loc='upper left',fontsize=8)

ax1.set_ylim(bottom=0,top=60)
ax2.set_ylim(bottom=0,top=100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AdmissionAgeSplit')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
# dfAdmDiffCorr
dfAdmGrouped = pd.DataFrame()
print(dfAdmDiffCorr.columns)
print(dfAdmDiffCorr.columns[0:2])
dfAdmGrouped['0-19'] = dfAdmDiffCorr.iloc[:,0:2].sum(axis=1) 
# print(dfAdmDiffCorr.columns[2:7])
# dfAdmGrouped['20-69'] = dfAdmDiffCorr.iloc[:,2:7].sum(axis=1) 
print(dfAdmDiffCorr.columns[2:4])
dfAdmGrouped['20-39'] = dfAdmDiffCorr.iloc[:,2:4].sum(axis=1) 
print(dfAdmDiffCorr.columns[4:7])
dfAdmGrouped['40-69'] = dfAdmDiffCorr.iloc[:,4:7].sum(axis=1) 
print(dfAdmDiffCorr.columns[7:])
dfAdmGrouped['70+'] = dfAdmDiffCorr.iloc[:,7:].sum(axis=1) 

dfAdmGrouped.tail()

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+'],
      dtype='object')
Index(['0-9', '10-19'], dtype='object')
Index(['20-29', '30-39'], dtype='object')
Index(['40-49', '50-59', '60-69'], dtype='object')
Index(['70-79', '80-89', '90+'], dtype='object')


,0-19,20-39,40-69,70+
233,1.000000,9.0,17.000000,24.000000
234,2.000000,8.0,18.000000,18.000000
235,2.333333,8.0,15.333333,17.333333
236,2.333333,8.0,15.333333,17.333333
237,2.333333,8.0,15.333333,17.333333


In [50]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

bgColor = 'lightgrey'
fig.patch.set_facecolor(bgColor)
ax1.set_facecolor(bgColor)
ax2.set_facecolor(bgColor)

allAgeCur = dfAdmGrouped.columns
newVals = dfAdmGrouped['0-19'].values
curVals = np.zeros(newVals.shape)
prevVals = np.zeros(newVals.shape)

# totCase = dfAdmGrouped.iloc[:,1:].sum(axis=1)
totCase = dfAdmGrouped.sum(axis=1)
totCaseRM = rnMean(totCase,7)

ax1.plot(rnTime(fullDates,7),totCaseRM,'k',linewidth=1,label='Total')

firstDateToShow = np.datetime64('2021-05-01')

cmap = cm.get_cmap('Spectral_r',len(allAgeCur)+1)
for k in range(0,len(allAgeCur)):
    ag = allAgeCur[k]
    newVals = dfAdmGrouped[ag].values

    curVals = curVals + newVals

    ax1.fill_between(rnTime(fullDates,7),rnMean(curVals,7),rnMean(prevVals,7),color=cmap(k),label=ag)
    
    ax2.fill_between(rnTime(fullDates,7),100*np.divide(rnMean(curVals,7),totCaseRM),100*np.divide(rnMean(prevVals,7),totCaseRM),color=cmap(k),label=ag)

    # ax2.text(firstDateToShow,(100/totCase[0])*(prevVals[0]),ag)
    # ax1.text(firstDateToShow,prevVals[0]+curVals[0]/4,ag)

    prevVals += newVals
    

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey',label=int(k==0)*'Weekend')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')

# ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='grey',label='Weekend') 
# ax1.set_xlim(left=fullDates[3])
# ax1.set_xlim(left=firstDateToShow)
# ax1.set_xlim(right=fullDates[-1])
ax1.set_xlim(left=np.datetime64('2021-05-19'))
ax1.set_xlim(right=np.datetime64('2021-11-02'))

ax1.set_ylabel('Antal nyindlæggelser\n(per dag)')
ax2.set_ylabel('Andel ud af alle nyindlæggelser [%]')

# ax1.legend(loc='upper left',fontsize=12)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax1.set_ylim(bottom=0,top=50)
ax2.set_ylim(bottom=0,top=100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AdmissionAgeSplit_grouped')
    
    
ax1.set_xlim(left=np.datetime64('2021-07-15'))
ax1.set_xlim(right=np.datetime64('2021-11-02'))

if saveFigures:
    plt.savefig('figs/AdmissionAgeSplit_grouped_zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
np.where(fullDatesAdm == np.datetime64('2021-05-19'))

(array([72], dtype=int64),)

In [29]:
fig,ax1 = plt.subplots()

# firstDataPoint = 80
firstDataPoint = np.where(fullDatesAdm == np.datetime64('2021-05-19'))[0][0]
dfToPlot = dfAdmDiffCorr.iloc[firstDataPoint:]
daysToUse = fullDates[firstDataPoint:]

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %B'), fontsize =10)
g.invert_yaxis()

plt.title('Daglige indlæggelser')

if saveFigures:
    plt.savefig('figs/HeatmapAdmissionsNew')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
# normDF = 100000 * dfAdmDiffCorr.div(DK_age)
normDF[normDF == 0] = 0

In [31]:
normDF = 100000 * dfAdmDiffCorr.div(DK_age)

normDF[normDF <= 0] = 0

fig,ax1 = plt.subplots()

# firstDataPoint = 28
dfToPlot = normDF.iloc[firstDataPoint:]
daysToUse = fullDates[firstDataPoint:]

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %B'), fontsize =10)
g.invert_yaxis()


plt.title('Daglige indlæggelser, per 100.000')

if saveFigures:
    plt.savefig('figs/HeatmapAdmissionsNormNew')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
# dfAdmDiffCorr

In [33]:

firstDataPoint = 65
dfToPlot = dfAdmDiffCorr.iloc[firstDataPoint:]
daysToUse = fullDates[firstDataPoint:]


dfAdmDiffCorrRM = pd.DataFrame()

meanWidth = 7
for curCol in dfAdmDiffCorr.columns:
    dfAdmDiffCorrRM[curCol] = rnMean(dfAdmDiffCorr[curCol],meanWidth)
# dfAdmDiffCorrRM[curCol]
dfToPlot = dfAdmDiffCorrRM.iloc[firstDataPoint:]

fig,ax1 = plt.subplots()

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %b'), fontsize =10)
g.invert_yaxis()

g.set_yticklabels(g.get_yticklabels(),rotation=0)

plt.title(f'Daglige indlæggelser, {meanWidth} dages gennemsnit')

if saveFigures:
    plt.savefig(f'figs/HeatmapAdmNew_{meanWidth}rm')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:

firstDataPoint = 65
dfToPlot = normDF.iloc[firstDataPoint:]
daysToUse = fullDates[firstDataPoint:]


normDFRM = pd.DataFrame()

meanWidth = 7
for curCol in normDF.columns:
    normDFRM[curCol] = rnMean(normDF[curCol],meanWidth)
# normDFRM[curCol]
dfToPlot = normDFRM.iloc[firstDataPoint:]

fig,ax1 = plt.subplots()

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %B'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %b'), fontsize =10)
g.invert_yaxis()

g.set_yticklabels(g.get_yticklabels(),rotation=0)

plt.title(f'Daglige indlæggelser per 100.000, {meanWidth} dages gennemsnit')

if saveFigures:
    plt.savefig(f'figs/HeatmapAdmNewNorm_{meanWidth}rm')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Dødsfald

In [35]:
# ssidatapath = "ssi_dashboard"
# rootdir = os.getcwd() +"\\" + ssidatapath


# for f in os.scandir(rootdir):
#     curPath = f.path
#     latestdir = curPath
#     latestDate = pd.to_datetime(curPath[-10:])
#     try: 
#         curdf = pd.read_csv(curPath+'\\Regionalt_DB\\05_bekraeftede_tilfaelde_doede_pr_region_pr_alders_grp.csv',delimiter=';',encoding='latin-1')
#     except:
#         2+2
    
    
# allDates = []

# ageGroups = curdf['Alders gruppe'].unique()
# fulldf = pd.DataFrame(columns=ageGroups)
# # fulldf

# curCounter = 0


# for f in os.scandir(rootdir):
#     curPath = f.path
#     latestdir = curPath
#     latestDate = pd.to_datetime(curPath[-10:])
    
#     try: 
#         curdf = pd.read_csv(curPath+'\\Regionalt_DB\\05_bekraeftede_tilfaelde_doede_pr_region_pr_alders_grp.csv',delimiter=';',encoding='latin-1')
#         curDeaths = curdf.groupby('Alders gruppe').sum()['Døde'].values
#         fulldf.loc[curCounter] = curDeaths
#         curCounter += 1
        
#         allDates.append(latestDate)
#     except:
#         print(curPath)
    

# print(latestdir)
# print(latestDate)
# # fulldf

In [36]:

firstDataPoint = np.where(fullDates == np.datetime64('2021-04-19'))[0][0]

In [37]:
dfDeathDiffCorr

# firstDataPoint = 30
firstDataPoint = np.where(fullDates == np.datetime64('2021-04-15'))[0][0]
dfToPlot = dfDeathDiffCorr.iloc[firstDataPoint:,:-1]
daysToUse = fullDates[firstDataPoint:]

dfDeathDiffCorrRM = pd.DataFrame()

meanWidth = 14
for curCol in dfDeathDiffCorr.columns[:-1]:
    dfDeathDiffCorrRM[curCol] = rnMean(dfDeathDiffCorr[curCol],meanWidth)
# dfDeathDiffCorrRM[curCol]
dfDeathDiffCorrRM[dfDeathDiffCorrRM < 0] = 0
dfToPlot = dfDeathDiffCorrRM.iloc[firstDataPoint:]


fig,ax1 = plt.subplots()

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %b'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %b'), fontsize =10)
g.invert_yaxis()

g.set_yticklabels(g.get_yticklabels(),rotation=0)


plt.title(f'Daglige dødsfald, {meanWidth} dages gennemsnit')
plt.title(f'Daily deaths, {meanWidth} day running mean')

if saveFigures:
    plt.savefig(f'figs/HeatmapDeathsNew_{meanWidth}rm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
normDFDeaths = 100000 * dfDeathDiffCorr.iloc[:,:-1].div(DK_age)
normDFDeaths[normDFDeaths < 0] = 0
normDFDeaths

# firstDataPoint = 30
firstDataPoint = np.where(fullDates == np.datetime64('2021-04-15'))[0][0]
# dfToPlot = normDFDeaths.iloc[firstDataPoint:]
daysToUse = fullDates[firstDataPoint:]

normDFDeathsRM = pd.DataFrame()

meanWidth = 14
for curCol in normDFDeaths.columns:
    normDFDeathsRM[curCol] = rnMean(normDFDeaths[curCol],meanWidth)
dfToPlot = normDFDeathsRM.iloc[firstDataPoint:]


fig,ax1 = plt.subplots()

g = sns.heatmap(dfToPlot.transpose(),xticklabels=pd.to_datetime(daysToUse).strftime('%d. %b'),cmap='turbo')
curXticks =  g.get_xticks()
daySkip = 7
g.set_xticks(curXticks[::daySkip])
g.set_xticklabels(pd.to_datetime(daysToUse[::daySkip]).strftime('%d. %b'), fontsize =10)
g.invert_yaxis()

plt.title(f'Daglige dødsfald per 100.000, {meanWidth} dages gennemsnit')
plt.title(f'Daily deaths per 100,000; {meanWidth} day running mean')

if saveFigures:
    plt.savefig(f'figs/HeatmapDeathsNewNorm_{meanWidth}rm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
# dfDeathDiffCorr.plot(x='Dato')

# Tests

In [40]:
# rootdir = ''
dashDir = 'ssi_data/'

rootdir = os.getcwd() +"\\" + dashDir


for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    latestDate = pd.to_datetime(curPath[-10:])

latestDate
latestdir

'c:\\Users\\rakrpe\\Documents\\PandemiX\\GithubRepos\\PandemiX\\DanskeData\\ssi_data/SSI_data_2021-11-01'

In [41]:
curFileName = '\\Cases_by_age.csv'
alldfs_test = []



allDates_test = []


for f in os.scandir(rootdir):
    curPath = f.path
    latestdir = curPath
    try:
        curDate = pd.to_datetime(curPath[-10:])

        
        curcsvpath = curPath + curFileName
        curdf = pd.read_csv(curcsvpath,delimiter = ';',dtype=str)

        allDates_test.append(curDate)
        alldfs_test.append(curdf)
    except:
        print(curPath)
    
print(f'Number of dataframes in alldfs_test: {len(alldfs_test)}')


Number of dataframes in alldfs_test: 379


In [42]:
# thisdf = alldfs_test[-1]
# int(thisdf[thisdf['Aldersgruppe'] == '0-9']['Antal_testede'].iloc[0].replace('.',''))
print(alldfs_test[-10])
# print(alldfs_test[-9])

# alldfs_test
# curdf

   Aldersgruppe Antal_bekræftede_COVID-19 Antal_testede Procent_positive
0           0-9                    31.071       487.133              6,4
1         10-19                    64.387       654.769              9,8
2         20-29                    78.875       743.204             10,6
3         30-39                    52.402       658.485              8,0
4         40-49                    51.436       698.567              7,4
5         50-59                    45.563       715.416              6,4
6         60-69                    23.713       544.533              4,4
7         70-79                    13.899       420.506              3,3
8         80-89                     6.765       159.123              4,3
9           90+                     2.048        33.275              6,2
10        I alt                   370.159     5.115.011              7,2


In [43]:
numDataframe = len(alldfs_test)

# allAge = alldfs_test[0]['Alders gruppe'].unique()

dfTest = pd.DataFrame()

dfTest['Dato'] = allDates_test


for ag in range(len(allAge)):
    curAge = allAge[ag]

    allTest = []

    prevDate = allDates_test[0]

    for da in range(0,numDataframe):
        thisdf = alldfs_test[da]
        thisdf = thisdf[thisdf['Aldersgruppe'] == curAge]

        thisTest = int(thisdf['Antal_testede'].iloc[0].replace('.',''))


        allTest.append(thisTest)


    
    dfTest[curAge] = allTest

In [44]:
# Calculate difference, and average out weekends

dfTestDiff = dfTest.diff().iloc[1:]

actualDiffDates = dfTestDiff.Dato
actualDiffDates = (actualDiffDates/np.timedelta64(1,'D')).astype(int)


fullDatesTest = np.arange(allDates_test[0],allDates_test[-1],np.timedelta64(1,'D'))

dfTestDiffCorr = dfTestDiff.copy()

for r in range(0,len(dfTestDiff)):
    if ((actualDiffDates.values[r]) != 1):
        # print(dfTestDiff.iloc[r])
        curRow = dfTestDiff.iloc[r]
        numDays = int(curRow.Dato/np.timedelta64(1,'D'))

        rowToAdd = curRow/numDays

        for k in range(1,numDays+1):
            dfTestDiffCorr.loc[r+(0.1*k)] = rowToAdd
        dfTestDiffCorr = dfTestDiffCorr.drop(r+1)
        


dfTestDiffCorr = dfTestDiffCorr.sort_index().reset_index(drop=True)
dfTestDiffCorr = dfTestDiffCorr.iloc[:,1:]

In [45]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

bgColor = 'lightgrey'
fig.patch.set_facecolor(bgColor)
ax1.set_facecolor(bgColor)
ax2.set_facecolor(bgColor)

newVals = dfTestDiffCorr['0-9'].values
curVals = np.zeros(newVals.shape)
prevVals = np.zeros(newVals.shape)

totCase = dfTestDiffCorr.sum(axis=1)
totCaseRM = rnMean(totCase,7)

ax1.plot(rnTime(fullDatesTest,7),totCaseRM,'k',linewidth=1,label='Total')

firstDateToShow = np.datetime64('2021-05-01')

cmap = cm.get_cmap('Spectral',len(allAge)+1)
for k in range(0,len(allAge)):
    ag = allAge[k]
    newVals = dfTestDiffCorr[ag].values

    curVals = curVals + newVals

    ax1.fill_between(rnTime(fullDatesTest,7),rnMean(curVals,7),rnMean(prevVals,7),color=cmap(k),label=ag)
    
    ax2.fill_between(rnTime(fullDatesTest,7),100*np.divide(rnMean(curVals,7),totCaseRM),100*np.divide(rnMean(prevVals,7),totCaseRM),color=cmap(k),label=ag)

    # ax2.text(firstDateToShow,(100/totCase[0])*(prevVals[0]),ag)
    # ax1.text(firstDateToShow,prevVals[0]+curVals[0]/4,ag)

    prevVals += newVals
    

# Draw weekends
firstSunday = np.datetime64('2021-01-03')
numWeeks = 52
for k in range(0,numWeeks):
    curSunday = firstSunday + np.timedelta64(7*k,'D')
    ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')
    ax2.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),zorder=-1,facecolor='grey')

ax1.axvspan(curSunday-np.timedelta64(1,'D')-np.timedelta64(12,'h'),curSunday+np.timedelta64(12,'h'),facecolor='lightgrey',label='Weekend') 
# ax1.set_xlim(left=fullDatesTest[3])
ax1.set_xlim(left=firstDateToShow)
ax1.set_xlim(right=fullDatesTest[-4])

ax1.set_ylabel('Antal nye førstegangstestede')
ax2.set_ylabel('Andel ud af alle test [%]')

ax1.legend(loc='upper left',fontsize=8)

ax1.set_ylim(bottom=0,top=5000)
ax2.set_ylim(bottom=0,top=100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%B'))
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/TestAgeSplit')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …